In [300]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))

from pokerkit import HandHistory
from data.utils.db import get_db_connection
from data.utils.hand_history import load_hhs_from_db

In [301]:
con = get_db_connection()
query = '''
with cte as (
    select
        p.name,
        count(distinct a.hand_id) as hands_with_shown_cards
    from actions a
    join players p on a.hand_id = p.hand_id
    and a.actor = concat('p', p.player_idx)
    where a.action_type == 'sm'
    and a.cards <> '????'
    group by p.name
    order by hands_with_shown_cards desc
)
select count(*) from cte;
'''
con.sql(query).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        19808 │
└──────────────┘



In [302]:
hand_id, = con.sql('select hand_id from hands limit 1;').fetchone()
hand_id

3031782172

In [303]:
load_hhs_from_db([hand_id], con)[0]

HandHistory(variant='NT', ante_trimming_status=False, antes=[Decimal('0.0'), Decimal('0.0'), Decimal('0.0'), Decimal('0.0'), Decimal('0.0')], blinds_or_straddles=None, bring_in=None, small_bet=None, big_bet=None, min_bet=Decimal('6.0'), starting_stacks=[Decimal('1310.4'), Decimal('2068.9'), Decimal('755.25'), Decimal('1462.85'), Decimal('1324.3')], actions=['d dh p1 ????', 'd dh p2 ????', 'd dh p3 ????', 'd dh p4 ????', 'd dh p5 ????', 'p3 f', 'p4 f', 'p5 f', 'p1 f'], author=None, event=None, url=None, venue='Absolute Poker', address=None, city=None, region=None, postal_code=None, country=None, time=None, time_zone=None, day=None, month=None, year=None, hand=None, level=None, seats=None, seat_count=None, table='GUMWOOD AVE', players=['sSZBJzpy1ZH4aGvCJdw9qw', 'QE2Vn7Qn2j+IUR+rOKncTA', '8z33KASCGgAHVfxu6HD4UA', 'yOOMTqsR9EFECI4lSNpp1A', 'dYzGQlQYQnkJl0uVsvJJIg'], finishing_stacks=[Decimal('NaN'), Decimal('NaN'), Decimal('NaN'), Decimal('NaN'), Decimal('NaN')], winnings=[Decimal('0.0'), 

In [304]:
con.close()